# **Character Code**
* 캐릭터 정보를 입력받아 캐릭터 코드를 반환받는 스크립트입니다.
* 쿼리 작성 후, 원하는 캐릭터 정보를 확인한 후 캐릭터 코드를 요청합니다.

### **Import**

In [ ]:
from concurrent.futures import ThreadPoolExecutor, as_completed
from src.api_request import DNF_API
from configs.config import MYSQL_CONNECTION_STRING
from sqlalchemy import create_engine
# from collections import defaultdict as dd
import pandas as pd

%load_ext sql
%sql {MYSQL_CONNECTION_STRING}
loader = DNF_API()
engine = create_engine(MYSQL_CONNECTION_STRING)

### **직업 정보 가져오기**
* job_info 테이블에서 원하는 직업 또는 직업군을 선택합니다.


In [ ]:
query = """
select *
from job_info
"""

job_info = %sql {query}
df_job_info = pd.DataFrame(job_info)
df_job_info

### **직업 데이터 가져오기**
* 크롤링을 통해 DB에 저장한 character_info 테이블에서 


In [ ]:
query = """
select *
from character_info
where job_name = '크리에이터'
"""
# where job_name in ('비질란테','헌터')

request = %sql {query}
request_list = list(request)
df_request = pd.DataFrame(request)
df_request.head()

### **캐릭터 코드 불러오기 - 멀티스레드**
* 요청받은 직업의 캐릭터 정보들을 이용해 캐릭터 코드를 불러옵니다.
* 속도를 높이기 위해 멀티스레드를 사용합니다.

In [17]:

# todo : api_request에 추가 또는 script화 예정

# * 성능 문제로 인해 iterrows(), apply lambda 등 사용 x
# df['char_code'] = df.apply(lambda row: char_code(row['sv_eng'], row['char_name_encoded']), axis=1)
# df.to_sql('new_character_info', con=engine, if_exists='replace', index=False)
# 분당 약 1000개

def request_char_code(sv_eng: str, char_name: str):
    
    """
    ### Summary
        - 캐릭터의 고유 id를 반환합니다.

    ### Args
        - sv_eng (str) : 캐릭터 서버 (영문)
        - char_name (str) : 캐릭터 이름 (인코딩 시 오류 감소)
        
    Returns:
        characterId (str) : 캐릭터 코드 반환. 에러 발생시 None
    """
    
    try:
        code = loader.character_search(sv_eng, char_name)
        return code['characterId'][0]
    except Exception as e:
        print(f"Error getting character code for {char_name}: {e}")
        return None


def character_code(request_list: list, thread_num: int) -> list:

    """
    ### Summary
        - 캐릭터 정보 (영문 서버, 캐릭터 이름)들을 입력받아 캐릭터 코드를 반환하는 함수

    ### Args
        - request_list (list[str,...,str]) : 요청할 캐릭터 정보
        
    Returns:
        - result (list) : 캐릭터 코드 정보
    """

    L = len(request_list)
    result = [None]*L

    with ThreadPoolExecutor(max_workers = thread_num) as executor:
        
        futures = []
        for idx,info in enumerate(request_list):
            sv_eng = info[1]
            char_name = info[-1]
            future = executor.submit(request_char_code,
                                    sv_eng, char_name)
            futures.append((future, idx))

        for future,idx in futures:
            try:
                code = future.result()
                result[idx] = code
                
                if idx%1000 == 0:
                    print(f"Processed {(idx/L)*100:.2f}% ({idx}/{L}) rows")
                    
            except Exception as e:
                result[idx] = None
                print(f"Error in future for row {idx}: {e}")
                
    print('complete')
    
    return result

In [15]:

# df_request['char_code'] = character_code(request_list,16)
# df_request.to_sql(f'db_name', con=engine,
#                   if_exists='append', index=False)